In [29]:
import pandas as pd
import numpy as np
import glob
import re
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from kmodes import kmodes, kprototypes
from datetime import datetime, timedelta

%reload_ext autoreload
%autoreload 2

#pd.set_option('display.max_rows', None)

## To-Do: 
### 1) Filter for diesel gensets within local resource area. 
### 2) Estimate HP for each genset using SCAQMD default emission rates? Might need guess of how many hours they run.
### 3) Figure out average emissions for each pollutant per HP. USe conversion from HP to KW to find avoided emissions per KW.

In [8]:
hp_to_kw = 0.7457

In [9]:
lbs_to_short_tons = 0.0005

In [10]:
diesel_gensets = pd.read_csv('data/SCAQMD_dieselBUGs_2020_PRR#1399752.csv')

In [11]:
pols = ['Emi Amt Av30 Co',
       'Emi Amt Av30 Nox', 'Emi Amt Av30 Pm10', 'Emi Amt Av30 Rog',
       'Emi Amt Av30 Sox', 'Emi Amt Av30 Tog']

In [12]:
for pol in pols:
    pol_name = pol.split(' ')[-1]
    diesel_gensets[pol_name + '(TPY)']= diesel_gensets[pol]* lbs_to_short_tons * 365

In [13]:
diesel_gensets['Total_pollution_summed'] = diesel_gensets['Co(TPY)']+diesel_gensets['Nox(TPY)']+diesel_gensets['Pm10(TPY)']+diesel_gensets['Rog(TPY)']+diesel_gensets['Sox(TPY)']+diesel_gensets['Tog(TPY)']


In [14]:
diesel_gensets_nonzero_emissions = diesel_gensets[diesel_gensets['Total_pollution_summed']>0]

In [15]:
tpy_cols = ['Co(TPY)', 'Nox(TPY)',
       'Pm10(TPY)', 'Rog(TPY)', 'Sox(TPY)', 'Tog(TPY)']

In [16]:
emissions_by_bcat_type = diesel_gensets_nonzero_emissions.groupby(['BCAT Description'])['Co(TPY)', 'Nox(TPY)',
       'Pm10(TPY)', 'Rog(TPY)', 'Sox(TPY)', 'Tog(TPY)','Total_pollution_summed'].sum()

<ipython-input-16-521ec6a5e165>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  emissions_by_bcat_type = diesel_gensets_nonzero_emissions.groupby(['BCAT Description'])['Co(TPY)', 'Nox(TPY)',


In [17]:
highest_emitting_bcat_types = emissions_by_bcat_type[emissions_by_bcat_type['Total_pollution_summed']>500]

In [18]:
diesel_bcat_types = emissions_by_bcat_type[emissions_by_bcat_type.index.str.contains('DIESEL')]

In [22]:
diesel_em_gen_names = ['I C E (>500 HP) EM ELEC GEN DIESEL','I C E (50-500 HP) EM ELEC GEN-DIESEL']

In [23]:
diesel_em_gens = diesel_gensets_nonzero_emissions[diesel_gensets_nonzero_emissions['BCAT Description'].isin(diesel_em_gen_names)]

In [32]:
diesel_em_gens['Facility Name'].unique()

array(['LA CITY, DEPT OF GEN SERVICES',
       'UNIVERSITY SO CALIFORNIA,HEALTH SCIENCES', 'LA CITY, HARBOR DEPT',
       ..., 'SANMINA CORPORATION', 'ORTEL, A DIVISION OF EMCORE',
       'LAMPS PLUS INC/ PACIFIC COAST LIGHTING'], dtype=object)